In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
import pytz
from astropy import *
import pandas as pd
from astropy import units as u
from astroplan import *
from astroplan import download_IERS_A
from astropy.table import Table

download_IERS_A()



Found a better dataset:
https://www.cosmos.esa.int/web/hipparcos/sample-tables-1

## Set up
Got observatory Location

Got data, all with names and locations

In [2]:
#Our Observatory
observatory = Observer.at_site('Palomar')

In [3]:
data2 = pd.read_csv('data2.csv')


In [4]:
#removing data without a name
#Couldn't get mask to work
mask = np.where(isinstance(data2['Name'],str))
mask
#Made my own mask since I couldn't get the one to work
othermask = list()
for i in range(len(data2['Name'])):
    if(isinstance(data2['Name'][i],str)):
        othermask.append(True)
    else:
        othermask.append(False)
data = data2[othermask]
data = data.reset_index()

In [5]:
len(data['Name']) # answer 39

39

In [6]:
#Dates will be October 15th through the 30th
#Find which objects will be visible
data

,index,HIP,HD,ra (deg),dec (deg),V mag,Mv =V + 5log(parallax in mas) -10,parallax in mas,std error on parallax,relative precision of distance,total proper motion (mas/yr),pm in ra (mas/yr),pm in dec (mas/yr),transverse velocity,in CNS3,Name
0,0,70890,NaN,217.449,-62.681,11.01,15.45,772.33,2.42,0.003,3852.99,-3775.64,768.16,23.65,*,alpha_Cen_C
1,1,71681,128621.0,219.914,-60.839,1.35,5.70,742.12,1.40,0.002,3724.12,-3600.35,952.11,23.79,*,alpha^2_Cen
2,2,71683,128620.0,219.920,-60.835,-0.01,4.34,742.12,1.40,0.002,3709.62,-3678.19,481.84,23.70,*,alpha^1_Cen
3,3,87937,NaN,269.454,4.668,9.54,13.24,549.01,1.58,0.003,10357.70,-797.84,10326.93,89.43,*,Barnard's_star
4,5,32349,48915.0,101.289,-16.713,-1.44,1.45,379.21,1.58,0.004,1339.42,-546.01,-1223.08,16.74,*,alpha_CMa
5,7,16537,22049.0,53.235,-9.458,3.72,6.18,310.75,0.85,0.003,976.61,-976.44,17.97,14.90,*,epsilon_Eri
6,10,104214,201091.0,316.712,38.741,5.20,7.49,287.13,1.51,0.005,5280.65,4155.10,3258.9,87.18,*,61_Cyg_A
7,11,37279,61421.0,114.827,5.228,0.40,2.68,285.93,0.88,0.003,1258.50,-716.57,-1034.58,20.86,*,alpha_CMi
8,12,104217,201092.0,316.717,38.734,6.05,8.33,285.42,0.72,0.003,5172.40,4107.40,3143.72,85.91,*,61_Cyg_B
9,16,108870,209100.0,330.823,-56.780,4.69,6.89,275.76,0.69,0.003,4703.94,3959.97,-2538.84,80.86,*,epsilon_Ind


## Moon Phases

Used the internet as a source for the moon: https://www.calendar-12.com/moon_calendar/2019/october

## Observation Capability

For October 15-30

In [7]:
## Initial Objects
my_targets = list()

#Making each object a targetable object using given data
for i in range(len(data["Name"])):
    #Getting coords for each target
    coords = SkyCoord(data['ra (deg)'][i]*u.deg, 
                      data['dec (deg)'][i]*u.deg,
                      frame='icrs')
    
    # Getting all the targets added to a list
    my_targets.append(FixedTarget(name=data["Name"][i], coord=coords))

In [8]:
#Time range we have been given
time_range = Time(["2019-10-15 00:00", "2019-10-30 23:59"])

In [21]:
#Some constraints to help determine if each is viable
con = [AltitudeConstraint(30*u.deg, 90*u.deg),
AirmassConstraint(2), AtNightConstraint.twilight_civil(),
       MoonIlluminationConstraint(0.4),MoonSeparationConstraint(min=.1*u.deg)
      ]
#Table for the 39 objects
observability_table(observer=observatory,
                   constraints=con,
                   targets=my_targets,time_range=time_range)


target name,ever observable,always observable,fraction of time observable
str14,bool,bool,float64
alpha_Cen_C,False,False,0.0
alpha^2_Cen,False,False,0.0
alpha^1_Cen,False,False,0.0
Barnard's_star,True,False,0.009114583333333334
alpha_CMa,True,False,0.053385416666666664
epsilon_Eri,True,False,0.11197916666666667
61_Cyg_A,True,False,0.07291666666666667
alpha_CMi,True,False,0.06510416666666667
61_Cyg_B,True,False,0.07291666666666667


In [10]:
#Subset of 10 objects
#Manually picked these out from the above table
small_targets = list()
small_targets.append(my_targets[38])
small_targets.append(my_targets[37])
small_targets.append(my_targets[36])
small_targets.append(my_targets[33])
small_targets.append(my_targets[3])
small_targets.append(my_targets[4])
small_targets.append(my_targets[5])
small_targets.append(my_targets[6])
small_targets.append(my_targets[7])
small_targets.append(my_targets[8])

In [27]:
#Some constraints to help determine if each is visible
con = [AltitudeConstraint(30*u.deg, 90*u.deg),
AirmassConstraint(2), AtNightConstraint.twilight_astronomical(),
       MoonIlluminationConstraint(0.95),MoonSeparationConstraint(min=10*u.deg)]
#Worst case scenario (First half of the period)
#Time range we have been given
time_range = Time(["2019-10-15 00:00", "2019-10-21 23:59"])
#Picked objects
observability_table(observer=observatory,
                   constraints=con,
                   targets=small_targets,time_range=time_range)


target name,ever observable,always observable,fraction of time observable
str14,bool,bool,float64
delta_Eri,True,False,0.02976190476190476
gamma_Lep,True,False,0.011904761904761904
chi^1_Ori,True,False,0.026785714285714284
mu_Her,True,False,0.017857142857142856
Barnard's_star,True,False,0.01488095238095238
alpha_CMa,True,False,0.008928571428571428
epsilon_Eri,True,False,0.03273809523809524
61_Cyg_A,True,False,0.03869047619047619
alpha_CMi,True,False,0.011904761904761904


In [28]:
#Some constraints to help determine if each is viable
con = [AltitudeConstraint(30*u.deg, 90*u.deg),
AirmassConstraint(2), AtNightConstraint.twilight_civil(),
       MoonIlluminationConstraint(0.10),MoonSeparationConstraint(min=1*u.deg)]
#Best case scenario (Latter half of the period)
#Time range we have been given
time_range = Time(["2019-10-21 00:00", "2019-10-30 23:59"])
#Picked objects
observability_table(observer=observatory,
                   constraints=con,
                   targets=small_targets,time_range=time_range)


target name,ever observable,always observable,fraction of time observable
str14,bool,bool,float64
delta_Eri,True,False,0.07083333333333333
gamma_Lep,True,False,0.058333333333333334
chi^1_Ori,True,False,0.09583333333333334
mu_Her,False,False,0.0
Barnard's_star,False,False,0.0
alpha_CMa,True,False,0.06458333333333334
epsilon_Eri,True,False,0.07083333333333333
61_Cyg_A,True,False,0.00625
alpha_CMi,True,False,0.07291666666666667


## Observation Capability

For November 15-30

In [13]:
#New Time range
time_range_nov = Time(["2019-11-15 00:00", "2019-11-30 23:59"])

In [35]:

# Early on in our window the moon will be bright (90%)
# Early section is from the 15 to the 23 (When the moon gets to about 10 percent)
time_range_nov = Time(["2019-11-15 00:00", "2019-11-23 23:59"])
con = [AltitudeConstraint(30*u.deg, 90*u.deg),
AirmassConstraint(2), AtNightConstraint.twilight_civil(),
       MoonIlluminationConstraint(0.90),MoonSeparationConstraint(min=.10*u.deg)]
#Table
observability_table(observer=observatory,
                   constraints=con,
                   targets=small_targets,time_range=time_range_nov)


target name,ever observable,always observable,fraction of time observable
str14,bool,bool,float64
delta_Eri,True,False,0.027777777777777776
gamma_Lep,True,False,0.013888888888888888
chi^1_Ori,True,False,0.037037037037037035
mu_Her,False,False,0.0
Barnard's_star,False,False,0.0
alpha_CMa,True,False,0.020833333333333332
epsilon_Eri,True,False,0.027777777777777776
61_Cyg_A,True,False,0.016203703703703703
alpha_CMi,True,False,0.02546296296296296
